In [2]:
from dotenv import load_dotenv
import os 

load_dotenv(override=True)
api = os.getenv("API2")
default_model = os.getenv("OPENAI_DEFAULT_MODEL")

from openai import OpenAI
client = OpenAI(api_key= api )

In [6]:
import json
from datetime import datetime


##########################################################################################
########################## step 1 tool로 정의할 function 정의하기 #########################
##########################################################################################

def get_time(timezone="Asia/Seoul"):
    # 간단한 시간 조회 함수
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"{timezone} 기준 현재 시간: {current_time}"
  
##########################################################################################
########################## step 2 tool 정의하기 ##########################################
##########################################################################################

# 여러 개의 도구 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "현재 시간을 알려주는 함수",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "시간대 (기본값: Asia/Seoul)",
                        "default": "Asia/Seoul"
                    }
                },
                "required": []
            }
        }
    }
]

##########################################################################################
########################## step 3 tool 불러오기 ##########################################
##########################################################################################

user_message = input("메시지를 입력하세요: ")
messages = [{"role": "user", "content": user_message}]
response = client.chat.completions.create(
        model=default_model,
        messages=messages,
        temperature=0.6,
        tools=tools,
        tool_choice="auto",
        stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)
    if chunk.choices[0].delta.tool_calls is not None:
        tool_calls = chunk.choices[0].delta.tool_calls
        for tool_call in tool_calls:
            if tool_call.function:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                if not function_args:
                    function_args = {}
                else:
                    function_args = json.loads(function_args)
                if function_name == "get_time":
                    result = get_time(**function_args)
                    print(f"\n{result}")


# 사용자 입력 초기화
user_message = ""

JSONDecodeError: Expecting value: line 1 column 1 (char 0)